In [51]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [52]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
from coco import *
from adapter_utils import *
from utils import *
from transformers.adapters import LoRAConfig

In [ ]:
import pandas as pd
import wandb

api = wandb.Api()

# Project is specified by <entity/project-name>

sweeps = ['y3ya5bpc', 'qb5zkdsx', 'l72dh1w7', 'kuvylfn5', 'e6rkfmy6', 'ucanqlze', 'ry0ss5vc', 'leziylwb', 'e63yxyoh']

summary_list, config_list, type_list, val_loss_list = [], [], [], []
for sweep in sweeps:
    s = api.sweep(f"read/read/sweeps/{sweep}")
    for run in s.runs:
        his = run.history()
        if 'CIDEr' in his:
            his = his[~his['CIDEr'].isna()]
            cider = max(his['CIDEr'].max(), run.summary['CIDEr'])
            val_loss = his['val_loss'].min()
            lr = run.config['learning_rate']
            
            summary_list.append(cider)
            config_list.append(lr)
            type_list.append(s.name)
            val_loss_list.append(val_loss)

runs_df = pd.DataFrame({
    "CIDEr": summary_list,
    "lr": config_list,
    "ft_type": type_list,
    "val_loss": val_loss_list
})

In [ ]:
runs_df.to_csv("runs.csv", index=False)

In [ ]:
pd.DataFrame([
    group.sort_values("CIDEr", ascending=False).iloc[0].to_dict()
    for _, group in runs_df.groupby("ft_type")
])


In [58]:
data_module = CocoDataModule(
    data_dir="/data/home/ngjhn/read/coco",
    ann_dir="/data/home/ngjhn/read/coco/annotations/",
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    split_file="/data/home/ngjhn/read/coco/dataset_coco.json",
    collate_fn=collate_fn,
    train_batch_size=32,
    val_batch_size=128)
peft_confg = RNNAdapterConf(
    outputs_scaling_factor=1.,
    input_dim=768,
    rnn_dim=256,
    num_layers=1,
    rnn_type="gru",
    bidirectional=True,
)
model = ImageCaptioningModel(
    vision_model_name_or_path="google/vit-base-patch16-224-in21k",
    text_model_name_or_path="bert-base-uncased",
    data_module=data_module,
    learning_rate=1e-4,
    peft_conf=LoRAConf(r=64, alpha=8),
)
# freeze(model.model)
# config = LoRAConfig(r=8, alpha=16)
# model.model.decoder.add_adapter("lora", config=config, set_active=True)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.9.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.5.crossattention.self.query.bias', 'bert.encoder.layer.5.crossattention.output.dense.weight', 'bert.encoder.layer.4.crossattention.output.dense.bias', 'bert.e

Enables lora tuning; num trainable params 4,718,592, ratio:0.034
loading annotations into memory...
Done (t=0.59s)
creating index...
index created!
loading annotations into memory...
Done (t=0.28s)
creating index...
index created!
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!


In [ ]:
batch = next(iter(data_module.train_dataloader()))

In [ ]:
# model.model.decoder.active_adapters = "lora"
model.training_step(batch, 0)

/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/core/module.py:411: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


{'loss': tensor(13.9929, grad_fn=<NllLossBackward0>)}

In [ ]:
trainer = pl.Trainer(
    max_epochs=40,
    devices=1,
    callbacks=[
        TQDMProgressBar(),
    ],
    default_root_dir="/data/home/ngjhn/read",
    num_sanity_val_steps=0,
    check_val_every_n_epoch=2,
)
trainer.fit(model, data_module)

/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


loading annotations into memory...
Done (t=0.51s)
creating index...
index created!
loading annotations into memory...
Done (t=0.30s)
creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | VisionEncoderDecoderModel | 224 M 
1 | val_loss  | AvgMetric                 | 0     
2 | test_loss | AvgMetric                 | 0     
--------------------------------------------------------
589 K     Trainable params
224 M     Non-trainable params
224 M     Total params
899.441   Total estimated model params size (MB)


Epoch 1: 100%|██████████| 2587/2587 [16:52<00:00,  2.55it/s, v_num=28, loss=2.240]


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1190: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1288: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(








































100%|██████████| 40/40 [00:50<00:00,  1.26s/it]


...

Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.9 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4

SPICE evaluation took: 37.29 s
SPICE: 0.054
Bleu_1: 0.294
Bleu_2: 0.189
Bleu_3: 0.119
Bleu_4: 0.074
METEOR: 0.097
ROUGE_L: 0.207
CIDEr: 0.102
SPICE: 0.054


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:77: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 128. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Process Process-41:
Process Process-42:
Process Process-43:
Process Process-44:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiproce

In [ ]:
trainer.test(model, datamodule=data_module)

You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

There are adapters available but none are activated for the forward pass.
/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1190: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/transformers/generation/utils.py:1288: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There are adapters av

loading annotations into memory...
Done (t=0.26s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
tokenization...


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Process Process-18:
Process Process-20:
Process Process-17:
Process Process-19:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/multiproce